In [2]:
# Full cleaning & analysis script
# Paste into Colab / local notebook and run.


import pandas as pd
import numpy as np
import ast, json, re
from datetime import datetime
import matplotlib.pyplot as plt

input_path = 'infra_projects_clean_final.csv'
output_path = 'infra_projects_clean_perfect.csv'
megaproj_path = 'future_major_projects_sample.csv'
report_plot_path = 'failure_by_delivery_method.png'

# ---- Helpers ----
def parse_numeric(x):
    if pd.isna(x):
        return np.nan
    x = str(x).strip()
    if x == '' or x.lower() in ['nan','none','null']:
        return np.nan
    # Remove currency symbols and commas
    x_clean = re.sub(r'[^\d.eE\-+]', '', x)
    try:
        return float(x_clean)
    except:
        m = re.search(r'[-+]?[0-9]*\.?[0-9]+([eE][-+]?\d+)?', x)
        if m:
            try:
                return float(m.group(0))
            except:
                return np.nan
        return np.nan

def clean_sector(x):
    if pd.isna(x): return 'Unknown'
    s = str(x).lower()
    if any(k in s for k in ['water','wastewater','sewer','drinking']):
        return 'Water'
    if any(k in s for k in ['road','highway','bridge','rail','metro','airport','port','transit']):
        return 'Transport'
    if any(k in s for k in ['power','energy','solar','wind','transmission','electric']):
        return 'Energy'
    if any(k in s for k in ['hospital','clinic','school','university','prison','social']):
        return 'Social Infrastructure'
    if any(k in s for k in ['data center','industrial','manufacturing','logistic','warehouse','factory']):
        return 'Industrial'
    s_clean = str(x).strip()
    if len(s_clean)>0:
        return s_clean.title()
    return 'Unknown'

def infer_project_type_from_row(row):
    desc = str(row.get('description','')).lower()
    sector = str(row.get('sector_main_clean','')).lower()
    text = desc + " " + sector
    if any(k in text for k in ['road','highway','bridge','tunnel','metro','rail']):
        return 'Transport'
    if any(k in text for k in ['water','sewer','wastewater','pipeline','dam','desal']):
        return 'Water'
    if any(k in text for k in ['power','solar','wind','transmission','energy']):
        return 'Energy'
    if any(k in text for k in ['hospital','clinic','school','university']):
        return 'Social Infrastructure'
    if any(k in text for k in ['data center','industrial','manufacturing','logistic','factory']):
        return 'Industrial'
    return 'Unknown'

def parse_timestamp_from_row(row):
    # Prefer 'timestamp' column if present
    if 'timestamp' in row and pd.notna(row['timestamp']) and str(row['timestamp']).strip():
        try:
            dt = pd.to_datetime(str(row['timestamp']), errors='coerce')
            if not pd.isna(dt):
                return dt.date().isoformat(), row.get('timestamp_label', 'timestamp')
        except:
            pass
    # Next, try a 'timestamps' JSON-like field
    if 'timestamps' in row and pd.notna(row['timestamps']) and str(row['timestamps']).strip():
        s = str(row['timestamps']).strip()
        d = None
        try:
            d = ast.literal_eval(s)
        except Exception:
            try:
                d = json.loads(s)
            except Exception:
                d = None
        if isinstance(d, dict):
            pref = ['project_start_date','estimated_start_date','publish_date','start_date']
            for k in pref:
                if k in d and d[k]:
                    try:
                        dt = pd.to_datetime(d[k], errors='coerce')
                        if not pd.isna(dt):
                            return dt.date().isoformat(), k
                    except:
                        continue
            for k,v in d.items():
                try:
                    dt = pd.to_datetime(v, errors='coerce')
                    if not pd.isna(dt):
                        return dt.date().isoformat(), k
                except:
                    continue
    return (np.nan, np.nan)

# ---- Load ----
df = pd.read_csv(input_path, dtype=str)
df.columns = [c.strip() for c in df.columns]

# Ensure project_id
if 'project_id' not in df.columns or df['project_id'].isnull().all():
    df.insert(0, 'project_id', range(1, len(df)+1))
else:
    df['project_id'] = df['project_id'].fillna('')
    missing_ids = df['project_id']==''
    if missing_ids.any():
        # fill only blanks
        start = df.shape[0] - missing_ids.sum() + 1
        df.loc[missing_ids, 'project_id'] = range(start, start + missing_ids.sum())

# Budget handling
possible_model_budget_cols = [c for c in df.columns if 'budget' in c.lower() and 'usd' in c.lower()]
possible_budget_cols = [c for c in df.columns if 'budget' in c.lower()]
if 'budget_raw' not in df.columns:
    candidates = [c for c in possible_budget_cols if c not in possible_model_budget_cols]
    df['budget_raw'] = df[candidates[0]] if candidates else np.nan

model_usd_col = possible_model_budget_cols[0] if possible_model_budget_cols else None
df['ml_budget_usd_model'] = df[model_usd_col] if model_usd_col else np.nan

df['budget_raw_num'] = df['budget_raw'].apply(parse_numeric)
df['ml_budget_usd_model_num'] = df['ml_budget_usd_model'].apply(parse_numeric)

def compute_budget_usd(row):
    if not pd.isna(row['ml_budget_usd_model_num']):
        return row['ml_budget_usd_model_num']
    currency = str(row.get('currency','')).upper()
    if currency == 'USD' and not pd.isna(row['budget_raw_num']):
        return row['budget_raw_num']
    if (not row.get('currency')) or str(row.get('currency')).strip()=='':
        if not pd.isna(row['budget_raw_num']) and row['budget_raw_num']>1000:
            return row['budget_raw_num']
    return np.nan

df['budget_usd_clean_calc'] = df.apply(compute_budget_usd, axis=1)

# If budget_usd_clean exists prefer numeric version of it
if 'budget_usd_clean' in df.columns:
    df['budget_usd_clean_num'] = df['budget_usd_clean'].apply(parse_numeric)
    df['budget_usd_clean_final'] = df['budget_usd_clean_num'].fillna(df['budget_usd_clean_calc'])
else:
    df['budget_usd_clean_final'] = df['budget_usd_clean_calc']

# Sector & type inference
df['sector_main_clean'] = df['sector_main'].apply(clean_sector) if 'sector_main' in df.columns else 'Unknown'
if 'project_type' in df.columns:
    df['project_type_clean'] = df['project_type'].fillna('Unknown').apply(lambda x: x if str(x).strip().lower()!='unknown' and str(x).strip()!='' else None)
    df['project_type_clean'] = df.apply(lambda r: infer_project_type_from_row(r) if (r['project_type_clean'] is None or str(r['project_type_clean'])=='None') else str(r['project_type_clean']).title(), axis=1)
else:
    df['project_type_clean'] = df.apply(infer_project_type_from_row, axis=1)

# Delivery method
proc_cols = [c for c in df.columns if 'procurement' in c.lower() or 'procure' in c.lower() or ('delivery' in c.lower() and 'method' in c.lower())]
if 'delivery_method' in df.columns:
    df['delivery_method_clean'] = df['delivery_method'].fillna('Unknown')
elif proc_cols:
    df['delivery_method_clean'] = df[proc_cols[0]].fillna('Unknown')
else:
    df['delivery_method_clean'] = 'Unknown'
df['delivery_method_clean'] = df['delivery_method_clean'].astype(str).str.strip().str.title().replace({'Nan':'Unknown','None':'Unknown'})

# Timestamp parsing
parsed = df.apply(parse_timestamp_from_row, axis=1)
df['timestamp_parsed'] = parsed.apply(lambda x: x[0])
df['timestamp_label_parsed'] = parsed.apply(lambda x: x[1])

# failed_or_problematic flag
status_cols = [c for c in df.columns if 'status' in c.lower()]
def determine_failed(row):
    ml_stat = None
    for c in status_cols:
        if 'ml.' in c.lower() or 'status' in c.lower():
            val = row.get(c)
            if pd.notna(val) and str(val).strip()!='':
                ml_stat = str(val).strip().lower()
                break
    if ml_stat is None and 'status' in row and pd.notna(row['status']):
        ml_stat = str(row['status']).strip().lower()
    if ml_stat:
        if any(k in ml_stat for k in ['failure','abandon','cancel','delayed','suspended','abandoned']):
            return 1
    if 'failed_or_problematic' in df.columns:
        try:
            v = int(float(row.get('failed_or_problematic',0)))
            if v==1:
                return 1
        except:
            pass
    return 0

df['failed_or_problematic_final'] = df.apply(determine_failed, axis=1)

# is_infrastructure_project boolean
if 'is_infrastructure_project' in df.columns:
    df['is_infrastructure_project_final'] = df['is_infrastructure_project'].fillna('False').astype(str).map(lambda x: x.strip().lower() in ['1','true','yes','y','t']).astype(bool)
else:
    infra_kw = ['road','bridge','water','sew','sewer','dam','pipeline','hospital','school','power','solar','wind','transmission','metro','rail','port','airport','tunnel','wastewater','treatment']
    df['is_infrastructure_project_final'] = df['description'].fillna('').str.lower().apply(lambda s: any(k in s for k in infra_kw))

# megaproject flag
df['budget_usd_clean_final'] = pd.to_numeric(df['budget_usd_clean_final'], errors='coerce')
df['is_megaproject_final'] = df['budget_usd_clean_final'].fillna(0) >= 500_000_000

# completeness score
def completeness_score(row):
    score = 0
    weights = {'project_name':10,'description':10,'country_name':10,'budget_usd_clean_final':20,'sector_main_clean':10,'project_type_clean':10,'delivery_method_clean':20,'url':10}
    for k,w in weights.items():
        v = row.get(k, None)
        if pd.notna(v) and str(v).strip()!='' and str(v).lower()!='unknown' and not (isinstance(v,float) and np.isnan(v)):
            score += w
    return min(100, score)

df['data_completeness_score'] = df.apply(completeness_score, axis=1)

# Prepare final DataFrame
final_df = pd.DataFrame()
final_df['project_id'] = df['project_id']
final_df['project_name'] = df.get('project_name', df.get('title', ''))
final_df['description'] = df.get('description', '')
final_df['country_name'] = df.get('country_name', '')
final_df['state_name'] = df.get('state_name', '')
final_df['city_name'] = df.get('city_name', '')
final_df['currency'] = df.get('currency', '')
final_df['budget_raw'] = df.get('budget_raw', '')
final_df['budget_raw_num'] = df.get('budget_raw_num', '')
final_df['budget_usd_clean'] = df['budget_usd_clean_final']
final_df['sector_main'] = df['sector_main_clean']
final_df['project_type'] = df['project_type_clean']
final_df['is_infrastructure_project'] = df['is_infrastructure_project_final']
final_df['is_megaproject'] = df['is_megaproject_final']
final_df['delivery_method'] = df['delivery_method_clean']
final_df['timestamp'] = df['timestamp_parsed']
final_df['timestamp_label'] = df['timestamp_label_parsed']
final_df['source'] = df.get('source','')
final_df['url'] = df.get('url','')
final_df['failed_or_problematic'] = df['failed_or_problematic_final']
final_df['data_completeness_score'] = df['data_completeness_score']

# Save outputs
final_df.to_csv(output_path, index=False)
final_df[final_df['is_megaproject'] == True].to_csv(megaproj_path, index=False)

# Failure analysis and plot
group = final_df.groupby('delivery_method').agg(total=('project_id','count'), failed=('failed_or_problematic','sum')).reset_index()
group['failure_rate'] = 100 * group['failed'] / group['total']
group_sorted = group.sort_values('failure_rate', ascending=False)

plt.figure(figsize=(10,6))
plt.bar(group_sorted['delivery_method'].astype(str), group_sorted['failure_rate'].fillna(0))
plt.xticks(rotation=45, ha='right')
plt.ylabel('Failure rate (%)')
plt.title('Failure Rate by Delivery Method')
plt.tight_layout()
plt.savefig(report_plot_path)
plt.close()

# Print summaries
print("Saved cleaned CSV to:", output_path)
print("Megaproject sample saved to:", megaproj_path)
print("Failure-by-delivery-method plot saved to:", report_plot_path)
print("\nFinal dataset shape:", final_df.shape)
print("\nMissing counts (final):\n", final_df.isnull().sum())
print("\nTop delivery methods by failure rate:\n", group_sorted.head(20).to_string(index=False))


Saved cleaned CSV to: infra_projects_clean_perfect.csv
Megaproject sample saved to: future_major_projects_sample.csv
Failure-by-delivery-method plot saved to: failure_by_delivery_method.png

Final dataset shape: (720, 21)

Missing counts (final):
 project_id                   0
project_name                 0
description                  0
country_name                 0
state_name                   0
city_name                    0
currency                     0
budget_raw                   0
budget_raw_num               0
budget_usd_clean             0
sector_main                  0
project_type                 0
is_infrastructure_project    0
is_megaproject               0
delivery_method              0
timestamp                    0
timestamp_label              0
source                       0
url                          0
failed_or_problematic        0
data_completeness_score      0
dtype: int64

Top delivery methods by failure rate:
 delivery_method  total  failed  failure_rate
   

In [3]:
df.drop(columns=['delivery_method'], inplace=True, errors='ignore')


In [4]:
df['is_infrastructure_project'].head(20)

0      True
1     False
2      True
3     False
4      True
5      True
6      True
7     False
8      True
9      True
10     True
11     True
12     True
13    False
14     True
15    False
16     True
17     True
18    False
19     True
Name: is_infrastructure_project, dtype: object

In [5]:
import re
import pandas as pd

df = df.copy()

# -------------------------------------------
# 1. PROJECT TYPE / SECTOR AUTO-FILL
# -------------------------------------------
def guess_project_type(text):

    if pd.isna(text):
        return None

    text = text.lower()

    keywords = {
        "transport": ["highway", "road", "metro", "rail", "bridge", "airport", "corridor", "bus"],
        "energy": ["solar", "wind", "thermal", "hydro", "power plant", "electricity", "grid"],
        "water": ["water", "sewage", "irrigation", "drainage", "pipeline"],
        "industrial": ["industrial", "logistics", "manufacturing", "industry"],
        "social": ["school", "hospital", "university", "housing", "health", "education"],
    }

    for sector, words in keywords.items():
        for w in words:
            if w in text:
                return sector

    return "unknown"


df["project_type_fixed"] = df["project_type"].fillna(
    df["description"].apply(lambda x: guess_project_type(str(x)))   
)

df["sector_main_fixed"] = df["sector_main"].fillna(df["project_type_fixed"])
    

# -----------------------------------------------------
# 2. DELIVERY METHOD AUTO-FILL (Mix ML + Regex)
# -----------------------------------------------------
def fix_delivery_method(row):
    
    mlcol = "ml.procurement_method_redo2.results.method"

    # 1. ML output use karo
    if mlcol in df.columns:
        if pd.notna(row[mlcol]) and row[mlcol] != "Unknown":
            return row[mlcol]

    # 2. Text-based extraction
    text = (str(row["description"]) + " " + str(row["project_name"])).lower()

    if "ppp" in text or "public private" in text:
        return "PPP"
    if "epc" in text:
        return "EPC"
    if "design-build" in text or "design build" in text:
        return "Design-Build"
    if "turnkey" in text:
        return "Turnkey"

    return "Unknown"


df["delivery_method_fixed"] = df.apply(fix_delivery_method, axis=1)


# -------------------------------------------------------
# 3. DATE PATTERN EXTRACTION
# -------------------------------------------------------
date_pattern = r"(20[0-4][0-9])"     # Matches years 2000–2049


def extract_year(text, label):
    if pd.isna(text):
        return None
    
    matches = re.findall(date_pattern, text)
    if not matches:
        return None
    
    # simple heuristic
    if "start" in label:
        return min(matches)
    if "complete" in label:
        return max(matches)

    return matches[0]


df["estimated_construction_start_date"] = df["description"].apply(
    lambda x: extract_year(str(x), "start")
)

df["estimated_completion_date"] = df["description"].apply(
    lambda x: extract_year(str(x), "complete")
)


# -------------------------------------------------------
# 4. FINAL ACTUAL COST (if described in text)
# -------------------------------------------------------
cost_pattern = r"(\$?\s?\d+(\,\d+)*\s?(million|billion)?)"

def extract_cost(text):
    if pd.isna(text):
        return None
    match = re.search(cost_pattern, text.lower())
    return match.group(0) if match else None

df["final_actual_cost"] = df["description"].apply(lambda x: extract_cost(str(x)))


# -------------------------------------------------------
# 5. FAILURE / RISK NOTES
# -------------------------------------------------------
def detect_risk_notes(text):
    if pd.isna(text):
        return None

    text = text.lower()

    keywords = ["delay", "delayed", "litigation", "court", "cancelled", "suspended", "overrun"]

    for k in keywords:
        if k in text:
            return k

    return None

df["risk_notes"] = df["description"].apply(lambda x: detect_risk_notes(str(x)))


# -------------------------------------------------------
# SAVE OUTPUT
# -------------------------------------------------------
df.to_csv("infra_projects_clean_enhanced.csv", index=False)
print("All missing-value enhancement complete!")


All missing-value enhancement complete!


In [6]:
df1 = pd.read_csv("infra_projects_clean_enhanced.csv")
df1.head(20)

,project_id,project_name,description,country_name,state_name,city_name,currency,budget_raw,budget_usd_clean,sector_main,...,is_infrastructure_project_final,is_megaproject_final,data_completeness_score,project_type_fixed,sector_main_fixed,delivery_method_fixed,estimated_construction_start_date,estimated_completion_date,final_actual_cost,risk_notes
0,1,Replace Forcemain at Bond’s Path; rehabilitate...,(1) Replacement of a section of sewer force ma...,Canada,Newfoundland and Labrador,Unknown,CAD,2.138559e+05,1.614237e+05,"Steel, Water Supply And Storage, Manufacturing...",...,True,False,80,Unknown,"Steel, Water Supply And Storage, Manufacturing...",Unknown,NaN,NaN,1,NaN
1,2,Geometric Safety Improvements - Removal of Cha...,Removal of right-turn channels and islands at ...,Canada,Ontario,Unknown,CAD,3.889100e+06,3.889100e+06,Government,...,False,False,70,Unknown,Government,Unknown,NaN,NaN,5,NaN
2,3,"Repairs at the Les Saules, Charlesbourg and Be...","Major repair work at the Les Saules, Charlesbo...",Canada,Quebec,Unknown,CAD,2.500000e+05,1.925892e+05,Government,...,True,False,70,Unknown,Government,Unknown,NaN,NaN,NaN,NaN
3,4,Accessibility Rehabilitation of Recreational C...,The project involves making the Recreation Cen...,Canada,Ontario,Unknown,CAD,2.310000e+05,1.719402e+05,Government,...,False,False,70,Unknown,Government,Unknown,NaN,NaN,995,NaN
4,5,Edmonton Trail Hotel Development,No Description,Canada,Alberta,Calgary,CAD,2.999999e+06,2.232989e+06,"Hotel, Mass Transit, Rail, Commercial",...,True,False,70,Unknown,"Hotel, Mass Transit, Rail, Commercial",Unknown,NaN,NaN,NaN,NaN
5,6,St. Thomas Elgin General Hospital,The project involved the construction of a thr...,Canada,Unknown,Unknown,CAD,6.330000e+07,5.732340e+07,"Commercial, Hospital, Government",...,True,False,80,Unknown,"Commercial, Hospital, Government",Unknown,NaN,NaN,"106,000",NaN
6,7,Paint Lake Provincial Park WW Lift Station & R...,Rehabilitation of an existing wastewater lift ...,Canada,Manitoba,Winnipeg,CAD,1.183164e+06,9.437224e+05,"Water Supply And Storage, Car Parking, Power G...",...,True,False,80,Unknown,"Water Supply And Storage, Car Parking, Power G...",Unknown,NaN,NaN,NaN,NaN
7,8,Le Market Condos Master Report,"D?un cote, 2 immeubles proposeront 220 unites ...",Canada,Quebec,Laval,CAD,1.000000e+00,7.976260e-01,"Residential, Commercial, Manufacturing (Indust...",...,False,False,80,Unknown,"Residential, Commercial, Manufacturing (Indust...",Unknown,NaN,NaN,2,NaN
8,9,2023 Winter Drain,The work to be done under this specification c...,Canada,Ontario,Strathroy,CAD,1.109660e+05,8.259532e+04,"Water Supply And Storage, Government, Waste Pr...",...,True,False,80,Unknown,"Water Supply And Storage, Government, Waste Pr...",Unknown,NaN,NaN,NaN,NaN
9,10,Driveway-Parking Lots Pavement Renovations,Scope - The Work of this Contract comprises th...,Canada,Alberta,Leduc,CAD,1.388000e+06,1.033130e+06,"Water Supply And Storage, Car Parking, Road, W...",...,True,False,80,Unknown,"Water Supply And Storage, Car Parking, Road, W...",Unknown,NaN,NaN,1,NaN


In [7]:
df1.isnull().sum()

project_id                             0
project_name                           0
description                            0
country_name                           0
state_name                             0
city_name                              0
currency                               0
budget_raw                             0
budget_usd_clean                       0
sector_main                            0
project_type                           0
is_infrastructure_project              0
is_megaproject                         0
timestamp                              0
timestamp_label                        0
source                                 0
url                                    0
failed_or_problematic                  0
ml_budget_usd_model                    0
budget_raw_num                         0
ml_budget_usd_model_num                0
budget_usd_clean_calc                  0
budget_usd_clean_num                   0
budget_usd_clean_final                 0
sector_main_clea

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 720 entries, 0 to 719
Data columns (total 40 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   project_id                         720 non-null    object 
 1   project_name                       720 non-null    object 
 2   description                        720 non-null    object 
 3   country_name                       720 non-null    object 
 4   state_name                         720 non-null    object 
 5   city_name                          720 non-null    object 
 6   currency                           720 non-null    object 
 7   budget_raw                         720 non-null    object 
 8   budget_usd_clean                   720 non-null    object 
 9   sector_main                        720 non-null    object 
 10  project_type                       720 non-null    object 
 11  is_infrastructure_project          720 non-null    object 

In [10]:
# ----- KEEP ONLY REQUIRED COLUMNS -----

final_columns = [
    # original required
    "project_id", "project_name", "description",
    "country_name", "state_name", "city_name",
    "currency", "budget_raw", "budget_raw_num",
    "budget_usd_clean", "sector_main", "project_type",
    "is_infrastructure_project", "is_megaproject",
    "delivery_method_fixed",    # final cleaned column
    "timestamp", "timestamp_label", "source", "url",
    "failed_or_problematic",

    # enriched new fields
    "estimated_construction_start_date",
    "estimated_completion_date",
    "final_actual_cost",
    "risk_notes"
]

# Filter only columns that exist in df
final_columns = [col for col in final_columns if col in df.columns]

df_final = df[final_columns].copy()

# SAVE
df_final.to_csv("infra_projects_FINAL_SUBMISSION.csv", index=False)

print("Clean FINAL dataset saved!")
print("Final shape:", df_final.shape)
print("Columns:", df_final.columns.tolist())


Clean FINAL dataset saved!
Final shape: (720, 24)
Columns: ['project_id', 'project_name', 'description', 'country_name', 'state_name', 'city_name', 'currency', 'budget_raw', 'budget_raw_num', 'budget_usd_clean', 'sector_main', 'project_type', 'is_infrastructure_project', 'is_megaproject', 'delivery_method_fixed', 'timestamp', 'timestamp_label', 'source', 'url', 'failed_or_problematic', 'estimated_construction_start_date', 'estimated_completion_date', 'final_actual_cost', 'risk_notes']
